In [ ]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [44.4 kB]
Hit:12 http://ppa.launchpad.net/cran/

In [ ]:
# Start Spark session
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType
spark = SparkSession.builder.appName("Challenge").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Personal_Care_Appliances_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   32114233|R1QX6706ZWJ1P5|B00OYRW4UE|     223980852|Elite Sportz Exer...|Personal_Care_App...|          5|            0|          0|   N|                Y|Good quality. Shi...|Exactly as descri...| 2015-08-31|
|         US|   18125776|R3QWMLJHIW6P37|B0000537JQ|     819771537|     Ezy Dose Weekly|Personal_Care_App

In [ ]:
# Create the vine_table. DataFrame
vine_df = df.select(['review_id','star_rating','helpful_votes','total_votes','vine','verified_purchase'])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1QX6706ZWJ1P5|          5|            0|          0|   N|                Y|
|R3QWMLJHIW6P37|          5|            0|          0|   N|                Y|
|R14Z1VR1N0Z9G6|          5|            1|          1|   N|                Y|
| R25ZRJL0GH0U0|          2|            0|          0|   N|                Y|
|R3837KYH7AZNIY|          4|            0|          1|   N|                Y|
|R2MN0QYCY6EVIV|          5|            0|          0|   N|                Y|
|R3AN2UJ1D42ZS0|          5|            0|          0|   N|                Y|
|R3U29ZLUWEEK4M|          5|            0|          0|   N|                Y|
|R16ZDMJJHK796C|          5|            0|          0|   N|                N|
| RRRDOEJZD1Y22|          4|            0|          0|   N|     

In [ ]:
# Filter data to new df to retrieve all rows where the total_votes count is equal to or greater than 20 to pick reviews that are more likely to be helpful and to avoid having division by zeros errors later on

total_votes_count_df = vine_df.filter("total_votes >= 20")
total_votes_count_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2PT2X8FMYFCV3|          5|           39|         41|   N|                Y|
| RGCUQSH80SYY7|          1|           36|         47|   N|                N|
|R3E6EI073KP0AL|          2|           21|         36|   N|                Y|
|R3MYTXW7B4Z6YW|          4|          687|        711|   N|                Y|
|R2WMF1IWN5ZYO4|          5|           18|         24|   N|                Y|
| ROX6L7VJFS2P7|          5|          339|        371|   N|                N|
| RRKAST7D4QTW8|          1|           20|         27|   N|                N|
|R2QBKTWBDPL8BF|          5|           65|         83|   N|                N|
|R3ULT9I0U79N8L|          5|           49|         57|   N|                N|
|R20LABVEB8O83Q|          3|           11|         23|   N|     

In [66]:
# Filter total_votes_count_df Dataframe to create a new dataframe where the number of helpful_votes divided by total_votes is equal to or greater than 50%

fifty_percent_df = total_votes_count_df.filter(total_votes_count_df["helpful_votes"]/total_votes_count_df["total_votes"] >= 0.5)
fifty_percent_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2PT2X8FMYFCV3|          5|           39|         41|   N|                Y|
| RGCUQSH80SYY7|          1|           36|         47|   N|                N|
|R3E6EI073KP0AL|          2|           21|         36|   N|                Y|
|R3MYTXW7B4Z6YW|          4|          687|        711|   N|                Y|
|R2WMF1IWN5ZYO4|          5|           18|         24|   N|                Y|
| ROX6L7VJFS2P7|          5|          339|        371|   N|                N|
| RRKAST7D4QTW8|          1|           20|         27|   N|                N|
|R2QBKTWBDPL8BF|          5|           65|         83|   N|                N|
|R3ULT9I0U79N8L|          5|           49|         57|   N|                N|
| RAC19YVDYSIKB|          5|          161|        178|   N|     

In [67]:
# Filter the fifty_percent_df Dataframe to create a new Dataframe that retrieves all the rows where a review was written as part of the Vine program (paid) vine =='Y'

paid_vine_df = fifty_percent_df.filter("vine == 'Y'")
paid_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3R2RHQWEKX5NL|          5|           81|         89|   Y|                N|
|R1RZ4JZO8DAP7H|          3|          200|        227|   Y|                N|
| RQ94HI0WM8KIG|          5|          174|        188|   Y|                N|
+--------------+-----------+-------------+-----------+----+-----------------+



In [69]:
# Retrieve all the rows where the review was not part of the Vine program (unpaid) 

unpaid_vine_df = fifty_percent_df.filter("vine == 'N'")
unpaid_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2PT2X8FMYFCV3|          5|           39|         41|   N|                Y|
| RGCUQSH80SYY7|          1|           36|         47|   N|                N|
|R3E6EI073KP0AL|          2|           21|         36|   N|                Y|
|R3MYTXW7B4Z6YW|          4|          687|        711|   N|                Y|
|R2WMF1IWN5ZYO4|          5|           18|         24|   N|                Y|
| ROX6L7VJFS2P7|          5|          339|        371|   N|                N|
| RRKAST7D4QTW8|          1|           20|         27|   N|                N|
|R2QBKTWBDPL8BF|          5|           65|         83|   N|                N|
|R3ULT9I0U79N8L|          5|           49|         57|   N|                N|
| RAC19YVDYSIKB|          5|          161|        178|   N|     

In [70]:
# Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for the two types of review (paid vs unpaid).

#Total number of reviews for paid reviews
paid_vine_df.count()


3

In [71]:
# The number of 5 star reviews for paid reviews
number_of_paid_5_star_ratings = paid_vine_df.filter('star_rating == 5')
number_of_paid_5_star_ratings.count()

2

In [72]:
# The percentage of 5-star reviews for paid reviews

total= paid_vine_df.count()
result=paid_vine_df.filter('star_rating == 5').count()

percentage_paid = (result/total)*100 
percentage_paid




66.66666666666666

In [73]:
# Total number of reviews for unpaid reviews
unpaid_vine_df.count()

3094

In [74]:
# The number of 5 star reviews for unpaid reviews
number_of_unpaid_5_star_ratings = unpaid_vine_df.filter('star_rating == 5')
number_of_unpaid_5_star_ratings.count()

1704

In [75]:
# The percentage of 5-star reviews for unpaid reviews
second_total= unpaid_vine_df.count()
second_result=unpaid_vine_df.filter('star_rating == 5').count()

percentage_unpaid = (second_result/second_total)*100 
percentage_unpaid

55.07433742727861